This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [26]:
using DrWatson

In [27]:
@quickactivate "RxInferThesisExperiments"

In [28]:
using RxInferThesisExperiments, Turing, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames, Logging

In [29]:
Turing.setprogress!(false)

[ Info: [Turing]: progress logging is disabled globally
[ Info: [AdvancedVI]: global PROGRESS is set as false


false

In [30]:
const bfolder = datadir("lds", "turing")

"/Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing"

In [31]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [32]:
# Include the model specification
include(srcdir("models", "turing", "rotating.jl"));

In [33]:
function run_benchmark(params)
    return with_logger(NullLogger()) do
        @unpack T, d, nsamples, seed = params

        environment = RotatingTracking(d; rng = StableRNG(seed))

        states, observations = rand(StableRNG(seed), environment, T);
        model    = rotating(observations, environment.A, environment.B, environment.P, environment.Q) 
        result   = run_inference_sampling(model; nsamples = nsamples, rng = StableRNG(seed));
        e_states = extract_posteriors(T, environment.d, result)
        amse     = compute_amse(states, e_states)

        benchmark_modelcreation = @benchmark rotating($observations, $(environment.A), $(environment.B), $(environment.P), $(environment.Q))


        benchmark_inference = @benchmark run_inference_sampling(model; nsamples = $nsamples, rng = StableRNG($seed)) setup=begin
            model = rotating($observations, $(environment.A), $(environment.B), $(environment.P), $(environment.Q))
            states, observations = rand(StableRNG($seed), $environment, $T);
        end

        emse = compute_emse(seed) do _seed
            local environment = RotatingTracking(d; rng = StableRNG(_seed))
            local states, observations = rand(StableRNG(_seed), environment, T);
            local model    = rotating(observations, environment.A, environment.B, environment.P, environment.Q) 
            local result   = run_inference_sampling(model; nsamples = nsamples, rng = StableRNG(_seed))
            local e_states = extract_posteriors(T, environment.d, result)
            return compute_amse(states, e_states)
        end

        output = @strdict T d nsamples seed states e_states observations amse emse benchmark_modelcreation benchmark_inference

        return output
    end
end

run_benchmark (generic function with 1 method)

In [34]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"    => [ 10, 20, 30, 100, 200 ],
    "d"    => [ 2, 3, 4 ],
    "nsamples" => [ 100, 200, 300 ],
    "seed" => [ 42 ]
));

In [35]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=30_d=2_nsamples=200_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=30_d=2_nsamples=200_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=100_d=2_nsamples=200_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=100_d=2_nsamples=200_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=200_d=2_nsamples=200_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=200_d=2_nsamples=200_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=10_d=2_nsamples=300_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users

[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=20_d=4_nsamples=100_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=20_d=4_nsamples=100_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=30_d=4_nsamples=100_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=30_d=4_nsamples=100_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=100_d=4_nsamples=100_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=100_d=4_nsamples=100_seed=42.jld2 saved.
[ Info: File /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing/T=200_d=4_nsamples=100_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/

In [36]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/RxInferThesisExperiments/data/lds/turing for result files.
[ Info: Added 45 entries.


Row,T,nsamples,amse,d,emse,seed,inference,creation
,Int64?,Int64?,Float64?,Int64?,Float64?,Int64?,Tuple…?,Tuple…?
1,10,100,2.98317,2,3.04118,42,"(4.33342e7, 4.52563e7, 0.0)","(1.334, 1.45602, 0.0)"
2,10,200,2.88397,2,3.01702,42,"(8.77616e7, 9.02746e7, 0.0)","(1.333, 1.45752, 0.0)"
3,10,300,2.85057,2,3.02124,42,"(1.03986e8, 1.0611e8, 0.0)","(1.334, 1.45524, 0.0)"
4,10,100,6.4205,3,5.10988,42,"(6.70168e7, 6.88439e7, 0.0)","(1.333, 1.4769, 0.0)"
5,10,200,6.30174,3,5.06847,42,"(1.59525e8, 1.65235e8, 2.37475e6)","(1.333, 1.45905, 0.0)"
6,10,300,6.19063,3,5.04094,42,"(1.82726e8, 1.84452e8, 2.48733e6)","(1.333, 1.46002, 0.0)"
7,10,100,8.60239,4,7.4698,42,"(1.02635e8, 1.08605e8, 0.0)","(1.333, 1.46503, 0.0)"
8,10,200,8.003,4,7.34556,42,"(2.55949e8, 2.57985e8, 2.18821e6)","(1.333, 1.45239, 0.0)"
9,10,300,8.21727,4,7.28292,42,"(2.78665e8, 2.87559e8, 2.56521e6)","(1.375, 1.51988, 0.0)"


# Versions

In [37]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 10 × Apple M2 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 2 on 6 virtual cores


In [38]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/RxInferThesisExperiments/Project.toml`
  [b5ca4192] AdvancedVI v0.2.3
  [6e4b80f9] BenchmarkTools v1.3.2
  [76274a88] Bijectors v0.12.5
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
  [31c24e10] Distributions v0.25.95
  [634d3b9d] DrWatson v2.12.5
  [442a2c76] FastGaussQuadrature v0.5.1
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [14197337] GenericLinearAlgebra v0.3.11
  [19dc6840] HCubature v1.5.1
  [7073ff75] IJulia v1.24.2
  [b964fa9f] LaTeXStrings v1.3.0
  [bdcacae8] LoopVectorization v0.12.159
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
  [91a5bcdd] Plots v1.38.15
  [92933f4c] ProgressMeter v1.7.2
  [37e2e3b7] ReverseDiff v1.14.6
  [86711068] RxInfer v2.11.1
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
⌅ [2913bbd2] StatsBase v0.33.21
  [4c63d2b9] StatsFuns v1.3.0
  [fce5fe82] Turing v0.25.1